In [1]:
import pandas as pd
TotalCoresAvailable=21000
def factorize(val):
    comb= [(i, val / i, val) for i in range(1, int(val**0.5)+1) if val % i == 0]
    perm=comb
    for i in comb:
        perm=perm+[(i[1], i[0], i[2])]
    return perm    

factors=[]
for i in range(0,TotalCoresAvailable+1):
    factors = factors + factorize(i)




In [2]:
# factors

In [3]:
df=pd.DataFrame(factors, columns=["Exec", "Cores", "TotalCoresUsed"])
df=df.drop_duplicates(["Exec", "Cores"])
df

,Exec,Cores,TotalCoresUsed
0,1.0,1.0,1
2,1.0,2.0,2
3,2.0,1.0,2
4,1.0,3.0,3
5,3.0,1.0,3
...,...,...,...
212415,210.0,100.0,21000
212416,200.0,105.0,21000
212417,175.0,120.0,21000
212418,168.0,125.0,21000


In [4]:
df.to_csv("factors.csv")

In [5]:
import numpy as np
import pandas as pd
VM=pd.read_csv("../data/VM_list.csv")[["Cores", "Memory"]]
factor_list=pd.read_csv("factors.csv")[["Exec", "Cores"]]
node_range = pd.DataFrame(range(1,201), columns = ["node_count"])
df1 = VM.merge(node_range, how='cross')
df2=df1.merge(factor_list, how='cross')
print(len(df2))
df2["TotalAvailableCores"] = df2.Cores_x * df2.node_count
df2["TotalCoresUsed"] = df2.Exec * df2.Cores_y
mask = df2.TotalCoresUsed <= df2.TotalAvailableCores
df2=df2[mask]
print(len(df2))
df2=df2.reset_index()[["Cores_x", "Memory", "node_count", "Exec", "Cores_y"]]
df2=df2.astype({"Cores_x":"int8", "Memory":"int16", "node_count":"int16","Exec":"int16", "Cores_y":"int16"})
df2.to_parquet("factors.parquet")

382096800
75219523
